In [371]:
import sys
sys.path.append("/usr/local/TSG/python")
import TasmanianSG
import numpy as np

In [395]:
grid = TasmanianSG.TasmanianSparseGrid()
iDim = 2
iLevel = 10

In [396]:
grid.makeGlobalGrid(iDim,0,iLevel,"level","chebyshev",[])
aPoints = grid.getPoints()
aWeights = grid.getQuadratureWeights()
reordering = np.argsort(aPoints[:,0])
aPoints = aPoints[reordering]
aWeights = aWeights[reordering]
print(len(aPoints))

257


In [394]:
np.dot(aWeights,np.power(aPoints,8)) - 2/9

array([ 0.])

In [318]:
theta = 1.0
mu = 0.5
sigma = 0.5
def f(x):
    return theta*(mu-x)

def g(x):
    if np.isscalar(x):
        return sigma
    else:
        return sigma*np.ones(x.shape)

In [319]:
ic = 0.1
h = 0.25
g2ic = np.square(g(ic))

# setting things up to integrate from -scaling to +scaling
scaling = 3.0
xgrid = scaling*aPoints
xweights = scaling*aWeights

# define initial condition function
def p(x):
    p = np.exp(-np.square(x - ic - f(ic)*h)/(2.0*g2ic*h))/np.sqrt(2.0*np.pi*g2ic*h)
    return(p)

In [320]:
# check normalization of initial condition
print(np.dot(xweights,p(xgrid)))

[ 1.]


In [321]:
p2 = np.zeros(len(aPoints))

In [322]:
for j in np.arange(len(aPoints)):
    x = xgrid[j]
    numer = np.square(xgrid + f(x + xgrid)*h)
    denom = 2.0*np.square(g(x + xgrid))*h
    part1 = np.exp(-numer/denom)/np.sqrt(np.pi*denom)
    part2 = p(x + xgrid)
    p2[j] = np.dot(xweights,part1*part2)

In [323]:
print(np.dot(xweights,p2))

1.0


In [324]:
def phi(x):
    return np.exp(-np.square(x)/2.0)/np.sqrt(2.0*np.pi)

In [325]:
amat = np.zeros((len(xweights),len(xweights)))
#endpt = scaling
#collocgrid = np.linspace(-endpt,endpt,len(aWeights))
#spacing = collocgrid[1]-collocgrid[0]
collocgrid = xgrid
spacing = np.mean(xgrid[1:len(xweights)] - xgrid[0:(len(xweights)-1)])
print(str(spacing) + "\n")
for j in np.arange(len(aPoints)):
    amat[:,j] = (phi((xgrid - collocgrid[j])/spacing)/spacing).flatten()
    nor = np.dot(amat[:,j],xweights)


0.06



In [366]:
# display condition number
print(np.linalg.cond(amat,p=2))

# store psuedoinverse
amatinv = np.linalg.pinv(amat,rcond=1.0e-8)

# coefficients corresponding to p2
c2 = np.matmul(amatinv,p2)

2.18879676003e+15


In [367]:
b = np.zeros(len(aPoints))
for l in np.arange(len(aPoints)):
    for k in np.arange(len(aPoints)):
        dg2h = 2.0*np.square(g(xgrid + xgrid[l]))*h
        t1 = np.power(np.pi*dg2h,-0.5)
        numer = np.square(xgrid[l] + f(xgrid + xgrid[l])*h)
        t2 = np.exp(-numer/dg2h)
        temp = xweights[l]*t1*t2*c2[k]*phi((xgrid + xgrid[l] - xgrid[k])/spacing)/spacing
        b += temp.flatten()
        
c3 = np.matmul(amatinv,b)

In [368]:
print(np.dot(xweights,b))

0.999976373392


In [369]:
np.sum(c3)

0.99940994000826244

In [370]:
np.sum(c2)

0.99940958268484437